In [1]:
import pandas as pd
import requests
import json
import re

## Definição das funções

In [2]:
def get_legislacao_ufam_txt():
    url = "https://raw.githubusercontent.com/menezs/support/main/legislacao_ufam/texto_juntos.txt"
    response = requests.get(url)
    texto = response.text
    
    return texto

In [3]:
def remover_espacos_extras(texto):
    palavras = texto.split()
    
    texto_limpo = ' '.join(palavras)
    
    return texto_limpo

In [4]:
def remover_caracteres_especificos(texto):
    padrao = r'["\-_.,º•·§!+]'
    texto_limpo = re.sub(padrao, '', texto)
    
    return texto_limpo

In [5]:
def extract_instruction_response_pairs(string: str):#-> Tuple[List[str], List[str]]:
    """
    Extracts pairs of instructions and responses from a JSON-formatted string.
    Parameters:
        - json_string (str): A string containing JSON-formatted instruction and response pairs.
    Returns:
        - instructions (list): A list of extracted instructions.
        - responses (list): A list of extracted responses corresponding to the instructions.
    """

    pattern = r'{"Pergunta": "(.*?)", "Resposta": "(.*?)"}'

    # Use re.findall to extract matches
    matches = re.findall(pattern, string)

    # Extract lists of "Instruction" and "Response"
    instructions = [match[0] for match in matches]
    responses = [match[1] for match in matches]

    return instructions, responses

In [6]:
def dividir_string(grande_string, tamanho_parte):
    """
    Divide uma string grande em partes menores.

    :param grande_string: A string que será dividida.
    :param tamanho_parte: O tamanho de cada parte.
    :return: Uma lista de partes menores da string.
    """
    partes = [grande_string[i:i + tamanho_parte] for i in range(0, len(grande_string), tamanho_parte)]
    return partes


In [7]:
def extract_instruction_response_pairs(string: str):#-> Tuple[List[str], List[str]]:
    """
    Extracts pairs of instructions and responses from a JSON-formatted string.

    Parameters:
        - json_string (str): A string containing JSON-formatted instruction and response pairs.

    Returns:
        - instructions (list): A list of extracted instructions.
        - responses (list): A list of extracted responses corresponding to the instructions.
    """
    texto = string.replace('\'', '"')

    pattern = r'\{"Pergunta":\s*"(.*?)",\s*"Resposta":\s*"(.*?)"\}'

    matches = re.findall(pattern, texto)

    instructions = [match[0] for match in matches]
    responses = [match[1] for match in matches]

    return instructions, responses

In [8]:
def verificar_repeticoes_perguntas(ins: list, res: list, All_instructions, All_reponses):

    for index in range(len(ins)):
        if ins[index] not in All_instructions:
            All_instructions.append(ins[index])
            All_reponses.append(res[index])
    

In [9]:
def criar_lista_dict(All_instructions, All_reponses):
    dict_full = []
    tam = len(All_instructions)
    if tam > 0:
        for index in range(tam):
            temp = {
                "Pergunta": All_instructions[index],
                "Resposta": All_reponses[index]
            }
            dict_full.append(temp)
        
    return dict_full

## Tratamento dos Dados

In [10]:
legislacao_full_txt = get_legislacao_ufam_txt()
legislacao_full_txt = legislacao_full_txt.split('\n')

In [11]:
texto_tratado = []
for frase in legislacao_full_txt:
    frase_formatada = remover_caracteres_especificos(frase)
    frase_formatada = remover_espacos_extras(frase_formatada)

    texto_tratado.append(frase_formatada)

In [12]:
legislacao_tratada = ' '.join(texto_tratado)

In [13]:
tamanho_parte = 900  # Defina o tamanho desejado para cada parte
partes = dividir_string(legislacao_tratada, tamanho_parte)

In [26]:
quant_partes = len(partes)
quant_partes

806

## Geração da base sintética de instruções

In [15]:
import time
# from openai import OpenAI
from langchain_openai import OpenAI
from tqdm import tqdm

In [16]:
# Utilizando o LM Studio
llm = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [17]:
start = time.time()

num_perguntas = 1

All_instructions = []
All_reponses = []

for idx in tqdm(range(quant_partes)):
    prompt =f"""### Gere somente uma pergunta com sua respectiva resposta sobre o seguinte texto:"{partes[idx]}"
    
    ### Você deve serguir o formato como no exemplo:{{'Pergunta': 'Exemplo de pergunta 1', 'Resposta': 'Exemplo de resposta 1'}}
    
    ### Você deve fornecer a pergunta e resposta no mesmo formato acima, sem nenhuma quebra de linha, sem repetição de pergunta, tenha a certeza de ter gerado somente um par pergunta-resposta, não gere código ou qualquer informação a mais que esteja fora do formato estabelecido.
    """
    
    resposta_llm = llm.generate([prompt])

    ins, res = extract_instruction_response_pairs(resposta_llm.generations[0][0].text)
    
    verificar_repeticoes_perguntas(ins, res, All_instructions, All_reponses)
    
print("\n\n===Time: {} seconds===".format(time.time()-start))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [09:31<00:00, 57.14s/it]



===Time: 571.4549188613892 seconds===


In [18]:
dict_full = criar_lista_dict(All_instructions, All_reponses)

In [19]:
dict_full

[{'Pergunta': 'Quais são as entidades que podem se candidatar à credenciamento?',
  'Resposta': 'Serão credenciadas pessoas jurídicas de direito público ou privado em funcionamento há 02 (dois) anos ou mais com ou sem fins lucrativos inclusive órgãos da Administração Pública direta, autárquica ou fundacional de qualquer dos Poderes da União, dos Estados, do Distrito Federal ou dos Municípios.'},
 {'Pergunta': 'Quais são os pré-requisitos para que uma entidade seja credenciada?',
  'Resposta': 'Serão credenciadas pessoas jurídicas de direito público ou privado em funcionamento há 02 (dois) anos ou mais com ou sem fins lucrativos.'},
 {'Pergunta': 'Qual é a Lei que regula o estágio obrigatório e não-obrigatório?',
  'Resposta': 'A Lei nº 11.788 de 25 de setembro de 2008.'},
 {'Pergunta': 'O número máximo de estagiário por entidade concedente será proporcional ao número do quadro de pessoal nos termos da Lei n 11788/2008.',
  'Resposta': 'I de 1 (um) a 5 (cinco) empregados: 1 (um) estagiá

## Salvar Base Sintética

In [20]:
nome_arquivo = 'base_sintética.json'

with open(nome_arquivo, 'w',encoding='utf-8') as arquivo:
    json.dump(dict_full, arquivo, ensure_ascii=False, indent=4)

print(f"Arquivo '{nome_arquivo}' salvo com sucesso!")

Arquivo 'base_sintética.json' salvo com sucesso!


## Verificar perguntas repetidas

In [21]:
with open(nome_arquivo, 'r', encoding='utf-8') as file:
    base_sintetica = json.load(file)


In [22]:
len(base_sintetica)

20

In [23]:
perguntas_unicas = {}

for item in base_sintetica:
    pergunta = item['Pergunta']
    if pergunta not in perguntas_unicas:
        perguntas_unicas[pergunta] = item

dados_sem_duplicatas = list(perguntas_unicas.values())
print(len(dados_sem_duplicatas))

20


In [24]:
with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
    json.dump(dados_sem_duplicatas, arquivo, ensure_ascii=False, indent=4)

print("Arquivo JSON atualizado com sucesso!")

Arquivo JSON atualizado com sucesso!


In [25]:
nome_arquivo

'base_sintética.json'